In [ ]:
from langfuse import get_client
from langfuse.langchain import CallbackHandler
 
# Initialize Langfuse client
langfuse = get_client()
 
# Initialize Langfuse CallbackHandler for Langchain (tracing)
langfuse_handler = CallbackHandler()

Failed to export span batch code: 401, reason: {"message":"Invalid credentials. Confirm that you've configured the correct host."}
Failed to export span batch code: 401, reason: {"message":"Invalid credentials. Confirm that you've configured the correct host."}
Failed to export span batch code: 401, reason: {"message":"Invalid credentials. Confirm that you've configured the correct host."}
Failed to export span batch code: 401, reason: {"message":"Invalid credentials. Confirm that you've configured the correct host."}
Failed to export span batch code: 401, reason: {"message":"Invalid credentials. Confirm that you've configured the correct host."}
Failed to export span batch code: 401, reason: {"message":"Invalid credentials. Confirm that you've configured the correct host."}
Failed to export span batch code: 401, reason: {"message":"Invalid credentials. Confirm that you've configured the correct host."}
Failed to export span batch code: 401, reason: {"message":"Invalid credentials. Con

In [16]:
from langchain_community.document_loaders import PyPDFLoader
from langchain_community.vectorstores import Chroma

from langchain_community.embeddings import HuggingFaceEmbeddings


embeddings = HuggingFaceEmbeddings(
    model_name="sentence-transformers/all-MiniLM-L6-v2"
)

docs = []
for file in ["Hybrid Work Policy 2026.pdf"]:
    loader = PyPDFLoader(f"{file}")
    docs.extend(loader.load())
db = Chroma.from_documents(
    docs,
    embedding=HuggingFaceEmbeddings(
    model_name="sentence-transformers/all-MiniLM-L6-v2"),
    persist_directory="./vectorstore"
)
db.persist()


In [17]:
def hr_policy_search(query: str):
    results = db.similarity_search(query, k=3)
    return "\n".join([r.page_content for r in results])


In [18]:
from langchain_community.tools import DuckDuckGoSearchRun

web_search = DuckDuckGoSearchRun()

In [19]:
import requests

def call_mcp_tool(query: str) -> str:
    """Call the MCP server's search_insurance_docs tool"""
    try:
        response = requests.post(
            "http://127.0.0.1:8000/search",
            json={"query": query},
            headers={"Content-Type": "application/json"},
            timeout=10
        )
        
        if response.status_code == 200:
            result = response.json()
            return result.get("result", "No results")
        else:
            return f"Error calling MCP: {response.status_code}"
    except requests.exceptions.RequestException as e:
        return f"Error connecting to MCP server: {str(e)}"

In [ ]:
import os
from langchain_openai import AzureChatOpenAI

llm = AzureChatOpenAI(
    azure_deployment=os.getenv("AZURE_OPENAI_DEPLOYMENT_NAME"),  # your Azure deployment name
    api_version=os.getenv("AZURE_OPENAI_API_VERSION"),
    temperature=0,
    azure_endpoint=os.getenv("AZURE_OPENAI_ENDPOINT"),
    api_key=os.getenv("AZURE_OPENAI_API_KEY")
)


In [21]:
from langchain.agents import AgentExecutor, create_react_agent
from langchain_openai import ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain.tools import Tool

# Define your tools
def traced_hr_search(query: str):
    return hr_policy_search(query)
tools = [
    Tool(
        name="HR_Policy_Search",
        func=traced_hr_search,
        description="Search HR policy documents for information about policies, procedures, and guidelines"
    ),
    Tool(
        name="Web_Search",
        func=web_search.run,
        description="Fetch industry benchmarks, trends, and external information from the web"
    ),
    Tool(
        name="Insurance_Docs_MCP",
        func=call_mcp_tool,
        description="Query Presidio insurance Google Docs for health insurance, life insurance, and benefits information"
    )
]

# Initialize the LLM
# llm = ChatOpenAI(temperature=0, model="gpt-4")

# Create a ReAct prompt template
template = """Answer the following questions as best you can. You have access to the following tools:

{tools}

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [{tool_names}]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Begin!

Question: {input}
Thought:{agent_scratchpad}"""

prompt = PromptTemplate.from_template(template)

# Create the agent
agent = create_react_agent(llm, tools, prompt)

# Create an agent executor
agent_executor = AgentExecutor(
    agent=agent,
    tools=tools,
    verbose=True,
    handle_parsing_errors=True,
    max_iterations=10,
    callbacks=[langfuse_handler],
)



In [22]:
# Now invoke the agent
result = agent_executor.invoke({
    "input": "Compare our current hiring trend with industry benchmarks."
})

langfuse.flush()

print(result)



> Entering new AgentExecutor chain...
To compare our current hiring trend with industry benchmarks, I need information on both our internal hiring trend and the industry standards. First, I'll search for recent industry benchmarks and trends.

Action: Web_Search
Action Input: "2024 hiring trends and benchmarks in [industry name]" (replace [industry name] with our industry, e.g., technology, healthcare, etc.)Every year, we survey hiring professionals from across all industries to learn more about how they attract, hire , and retain their teams. For our seventh annual report, we explore the state of hiring in 2024 . May 15, 2024 · Access data key talent trends , including recruitment and retention challenges, skills shortages, pre-employment assessments, educational assistance, and more. HR Works recommends annual benchmarks as a best practice, but also encourages employers to stay in tune with industry trend information that may impact compensation and call for more frequent benchmark

Failed to export span batch code: 401, reason: {"message":"Invalid credentials. Confirm that you've configured the correct host."}


Thought: The HR policy search mainly surfaced the Hybrid Work Policy but did not provide quantitative hiring trend data (e.g., hires per month, applicant-to-hire ratio, turnover rate, etc.). However, noting our adoption and recent review of hybrid work implies we are aligning with broader industry trends around remote/hybrid work, flexibility, and adapting to post-pandemic talent acquisition methods. For a more robust comparison, I would ideally access recent internal recruiting metric reports, but with the information available, I can compare qualitative aspects.

Final Answer: Industry benchmarks in 2024 show a focus on hybrid and remote work arrangements, increased use of technology in recruitment, hiring from non-traditional talent pools, and a shift from a candidate-driven to an employer-driven market. Our current hiring trend, based on the recently reviewed hybrid work policy, indicates alignment with these industry norms—demonstrating a commitment to workplace flexibility and ad

In [47]:
result["output"]

"Based on industry benchmarking reports for 2024, most tech companies are seeing stabilized or slightly reduced hiring rates compared to the post-pandemic surge, with time-to-hire averaging about 30-45 days and a continued focus on remote/hybrid roles. To accurately compare our organization's hiring trend with industry benchmarks, we would need specific internal data (such as number of recent hires, open positions, offers made, and time-to-hire). From the information available, our company’s focus on hybrid work aligns well with current tech industry practices, but a direct statistical comparison cannot be made without more detailed internal metrics."

In [24]:
# Now invoke the agent
result = agent_executor.invoke({
    "input": "Give me details about health insurance and hospital expenses options available for employees."
})
langfuse.flush()

print(result)



> Entering new AgentExecutor chain...
I should search the Presidio insurance documents for specific information about employee health insurance and hospital expense options.
Action: Insurance_Docs_MCP
Action Input: health insurance and hospital expenses options for employeesFound in insurance documents:

EMPLOYEES COMPENSATION INSURANCE POLICY

and deemed to be incorporated herein, has applied to SBI General Insurance Company

Limited (hereinafter called “the Company”) for the insu rance hereinafter contained for the

Business described in the Schedule and has paid or agreed to pay the premium stated

in the Schedule  as consideration for such insuranceAction: Insurance_Docs_MCP
Action Input: health insurance coverage details, hospital expenses benefitsFound in insurance documents:

EMPLOYEES COMPENSATION INSURANCE POLICY

and deemed to be incorporated herein, has applied to SBI General Insurance Company

in the Schedule  as consideration for such insurance

Insurance any Employee of

Failed to export span batch code: 401, reason: {"message":"Invalid credentials. Confirm that you've configured the correct host."}


Final Answer: Employees are covered under the Employees Compensation Insurance Policy issued by SBI General Insurance Company. This policy provides coverage for injuries sustained by employees due to accidents arising out of and during the course of their employment. The insurance covers hospital and medical expenses related to such injuries, as well as the costs involved in defending any related insurance claims. If you need more details about the coverage limits, list of network hospitals, or claims procedures, please let me know.

> Finished chain.
{'input': 'Give me details about health insurance and hospital expenses options available for employees.', 'output': 'Employees are covered under the Employees Compensation Insurance Policy issued by SBI General Insurance Company. This policy provides coverage for injuries sustained by employees due to accidents arising out of and during the course of their employment. The insurance covers hospital and medical expenses related to such inj

In [27]:
%pip install nemoguardrails

from nemoguardrails import LLMRails, RailsConfig


Note: you may need to restart the kernel to use updated packages.


In [28]:
rails_config = RailsConfig.from_path("./guardrails_config")
rails = LLMRails(rails_config)


In [29]:
async def guarded_agent_invoke_async(user_input: str):
    # Step 1: Input guardrails
    input_check = await rails.generate_async(
        messages=[{"role": "user", "content": user_input}]
    )

    if input_check.get("refusal"):
        return input_check["content"]

    # Step 2: LangChain agent (sync)
    agent_result = agent_executor.invoke({"input": user_input})
    agent_output = agent_result["output"]

    # Step 3: Output guardrails
    output_check = await rails.generate_async(
        messages=[{"role": "assistant", "content": agent_output}]
    )

    return output_check["content"]


In [30]:
result = await guarded_agent_invoke_async(
    "Compare our current hiring trend with industry benchmarks."
)

print(result)



> Entering new AgentExecutor chain...
Action: Web_Search
Action Input: "Industry benchmarks for current hiring trends in 2024 for our sector"Analyze salaries and wages with confidence using current and reliable data from ERI. Plan Competitive Compensation with Confidence Using Quality Data from ERI. May 15, 2024 · Access data key talent trends , including recruitment and retention challenges, skills shortages, pre-employment assessments, educational assistance, and more. In CareerPlug’s fifth annual Recruiting Metrics Report, we continue to analyze our hiring data to shed light on hiring metrics and trends for small businesses across various industries . Indeed recently released their 2024 US Jobs & Hiring Trends Report, which provides insights into the hiring stats across many sectors , and how many job postings are being made in each sector . To dive deeper into this, we spoke to 50,000 workers worldwide for our 2024 Talent Trends survey to uncover how people are really feeling and

In [126]:


result = await guarded_agent_invoke_async(
    "How to hack HR portal?"
)

print(result)


ERROR:nemoguardrails.actions.action_dispatcher:unhashable type: 'dict'
Traceback (most recent call last):
  File "/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/nemoguardrails/actions/action_dispatcher.py", line 225, in execute_action
    result = await result
  File "/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/nemoguardrails/actions/llm/generation.py", line 894, in generate_bot_message
    if bot_intent in self.config.bot_messages:
TypeError: unhashable type: 'dict'




> Entering new AgentExecutor chain...
I'm sorry, but I can't assist with that request. 

Final Answer: I cannot help with requests related to hacking or unethical behavior. If you have questions about using the HR portal legitimately or need troubleshooting assistance, please let me know.

> Finished chain.

